In [6]:
import os
from keras import Model
from keras.backend import categorical_crossentropy
from keras.layers import Dense, Softmax
from keras.optimizers import Adam
from keras_preprocessing.image import ImageDataGenerator
from models.AlexNetVAE import AlexNetVAE

model = AlexNetVAE(input_dim=(224,224,3), log_dir="./logs", z_dim=2000, kernel_visualization_layer=1, use_batch_norm=True, use_dropout=True, dropout_rate=0.3, feature_map_layers=[3], num_samples=5)
model.load_weights('../results/alexnet_vae_zdim_2000/weights.h5')
model = model.encoder
for layer in model.layers:
    layer.trainable = False
model_input = model.inputs
encoder_output = model.outputs
dense = Dense(1000)(encoder_output[0])
model_output = Softmax()(dense)

classifier = Model(model_input, model_output)

optimizer = Adam(lr=0.0001)
classifier.compile(optimizer=optimizer, loss=categorical_crossentropy, metrics=['accuracy'])

data_gen = ImageDataGenerator(rescale=1. / 255)
training_data = data_gen.flow_from_directory(
            directory=os.path.join('/Users/leo/Documents/Tesla/data', 'imagenet/ILSVRC/Data/CLS-LOC/train'),
            target_size=(224,224), batch_size=32,
            class_mode='categorical', interpolation='lanczos',
            follow_links=True)


Found 1281167 images belonging to 1000 classes.


In [1]:
import math
steps_per_epoch = math.ceil(training_data.n / 32)
classifier.fit_generator(training_data, shuffle=True, epochs=100, initial_epoch=0,steps_per_epoch=steps_per_epoch)


NameError: name 'training_data' is not defined